In [1]:
import os
import sys
sys.path.append(os.path.join(os.getcwd(), '..','..','..','..'))
from configs import spark_config as config
from utils import spark_utils as utils

# Create a Spark session
spark = utils.create_spark_session("fielding", {
    'spark.executor.memory': '3g',
    'spark.executor.cores': '6',
})

fielding_data = utils.load_data(spark,config.RAW_DATA_DIR, 't20_fielding_stats.csv')

fielding_data.show(5)

/home/ravikumar/miniconda3/envs/t20i/lib/python3.12/site-packages/airflow/configuration.py:859 FutureWarning: section/key [core/sql_alchemy_conn] has been deprecated, you should use[database/sql_alchemy_conn] instead. Please update your `conf.get*` call to use the new name

[2024-11-24T14:14:32.688+0530] {spark_utils.py:17} INFO - Creating Spark session.


your 131072x1 screen size is bogus. expect trouble
24/11/24 14:14:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/11/24 14:14:37 WARN Utils: spark.executor.instances less than spark.dynamicAllocation.minExecutors is invalid, ignoring its setting, please update your configs.


[2024-11-24T14:14:38.060+0530] {spark_utils.py:37} INFO - Spark session created successfully.
[2024-11-24T14:14:38.063+0530] {spark_utils.py:46} INFO - Loading data from t20_fielding_stats.csv.


+------------------+---+----+---+---+---+-----+-----+---+---+-------+
|            Player|Mat|Inns|Dis| Ct| St|Ct Wk|Ct Fi| MD|D/I| Season|
+------------------+---+----+---+---+---+-----+-----+---+---+-------+
|     AR Adams (NZ)|  1|   1|  0|  0|  0|    0|    0|  0|0.0|2004/05|
|    CL Cairns (NZ)|  1|   1|  0|  0|  0|    0|    0|  0|0.0|2004/05|
|   MJ Clarke (AUS)|  1|   1|  0|  0|  0|    0|    0|  0|0.0|2004/05|
|   SP Fleming (NZ)|  1|   1|  0|  0|  0|    0|    0|  0|0.0|2004/05|
|AC Gilchrist (AUS)|  1|   1|  0|  0|  0|    0|    0|  0|0.0|2004/05|
+------------------+---+----+---+---+---+-----+-----+---+---+-------+
only showing top 5 rows



In [2]:
fielding_data = fielding_data.select(['Player',"Mat","Inns","Dis","Ct","St","D/I","Season"]).sort(["Player","Season"])
fielding_data.show(5)

+----------------+---+----+---+---+---+-----+-------+
|          Player|Mat|Inns|Dis| Ct| St|  D/I| Season|
+----------------+---+----+---+---+---+-----+-------+
|A Ahmadhel (BUL)|  3|   3|  0|  0|  0|  0.0|2019/20|
|A Ahmadhel (BUL)|  1|   1|  0|  0|  0|0.000|   2020|
|A Ahmadhel (BUL)|  2|   2|  0|  0|  0|0.000|2020/21|
|A Ahmadhel (BUL)|  3|   3|  0|  0|  0|  0.0|   2021|
|A Ahmadhel (BUL)|  2|   1|  0|  0|  0|  0.0|   2023|
+----------------+---+----+---+---+---+-----+-------+
only showing top 5 rows



In [3]:
from pyspark.sql.functions import col, when
fielding_data = fielding_data.withColumn('Inns', when(col('Inns') == '-', '0').otherwise(col('Inns')).cast('float'))
fielding_data = fielding_data.withColumn('Dis', when(col('Dis') == '-', '0').otherwise(col('Dis')).cast('float'))
fielding_data = fielding_data.withColumn('Ct', when(col('Ct') == '-', '0').otherwise(col('Ct')).cast('float'))
fielding_data = fielding_data.withColumn('St', when(col('St') == '-', '0').otherwise(col('St')).cast('float'))
fielding_data = fielding_data.withColumn('D/I', when(col('D/I') == '-', col('Dis')/col('Inns')).otherwise(col('D/I')).cast('float')).fillna(0)
fielding_data.show(5)

+----------------+---+----+---+---+---+---+-------+
|          Player|Mat|Inns|Dis| Ct| St|D/I| Season|
+----------------+---+----+---+---+---+---+-------+
|A Ahmadhel (BUL)|  3| 3.0|0.0|0.0|0.0|0.0|2019/20|
|A Ahmadhel (BUL)|  1| 1.0|0.0|0.0|0.0|0.0|   2020|
|A Ahmadhel (BUL)|  2| 2.0|0.0|0.0|0.0|0.0|2020/21|
|A Ahmadhel (BUL)|  3| 3.0|0.0|0.0|0.0|0.0|   2021|
|A Ahmadhel (BUL)|  2| 1.0|0.0|0.0|0.0|0.0|   2023|
+----------------+---+----+---+---+---+---+-------+
only showing top 5 rows



In [4]:
from pyspark.sql.functions import regexp_extract, regexp_replace

# Extract the country name between '(' and ')'
fielding_data = fielding_data.withColumn("Country", regexp_extract(col("Player"), r"\((.*?)\)", 1))

# Extract the player's name before the first '('
fielding_data = fielding_data.withColumn("Player", regexp_extract(col("Player"), r"^(.*?)\s\(", 1))

# Show the result
fielding_data.show(5)

+----------+---+----+---+---+---+---+-------+-------+
|    Player|Mat|Inns|Dis| Ct| St|D/I| Season|Country|
+----------+---+----+---+---+---+---+-------+-------+
|A Ahmadhel|  3| 3.0|0.0|0.0|0.0|0.0|2019/20|    BUL|
|A Ahmadhel|  1| 1.0|0.0|0.0|0.0|0.0|   2020|    BUL|
|A Ahmadhel|  2| 2.0|0.0|0.0|0.0|0.0|2020/21|    BUL|
|A Ahmadhel|  3| 3.0|0.0|0.0|0.0|0.0|   2021|    BUL|
|A Ahmadhel|  2| 1.0|0.0|0.0|0.0|0.0|   2023|    BUL|
+----------+---+----+---+---+---+---+-------+-------+
only showing top 5 rows



In [5]:
from pyspark.sql import Window
from pyspark.sql.functions import col, sum as spark_sum, when, row_number, round

# Define the window specification for cumulative calculations
window_spec = (
    Window.partitionBy("Player", "Country")
    .orderBy("Season")
    .rowsBetween(Window.unboundedPreceding, -1)
)

# Window for row number to identify the first row per player and country
row_num_window = Window.partitionBy("Player", "Country").orderBy("Season")

# Perform cumulative calculations with conditions
fielding_data = (
    fielding_data.withColumn("row_num", row_number().over(row_num_window))
    .withColumn(
        "Cumulative Mat",
        when(col("row_num") == 1, 0).otherwise(spark_sum("Mat").over(window_spec)),
    )
    .withColumn(
        "Cumulative Inns",
        when(
            col("row_num") == 1, 0
        ).otherwise(  # Set 0 for the first row (before any match)
            spark_sum("Inns").over(window_spec)
        ),
    )
    .withColumn(
        "Cumulative Dis",
        when(
            col("row_num") == 1, 0
        ).otherwise(  # Set 0 for the first row (before any match)
            spark_sum("Dis").over(window_spec)
        ),
    )
    .withColumn(
        "Cumulative Ct",
        when(col("row_num") == 1, 0).otherwise(spark_sum("Ct").over(window_spec)),
    )
    .withColumn(
        "Cumulative St",
        when(col("row_num") == 1, 0).otherwise(spark_sum("St").over(window_spec)),
    )
    .withColumn(
        "Cumulative D/I",
        when(col("row_num") == 1, 0).otherwise(
            round(
                when(
                    spark_sum("Inns").over(window_spec) != 0,
                    spark_sum(("Dis")).over(window_spec)
                    / spark_sum("Inns").over(window_spec),
                ).otherwise(0),
                2,
            )
        ),
    )
    .drop("row_num")
)  # Drop the temporary row number column

# Show the resulting DataFrame
fielding_data.show(10)

+--------------+---+----+---+---+---+-----+-------+-------+--------------+---------------+--------------+-------------+-------------+--------------+
|        Player|Mat|Inns|Dis| Ct| St|  D/I| Season|Country|Cumulative Mat|Cumulative Inns|Cumulative Dis|Cumulative Ct|Cumulative St|Cumulative D/I|
+--------------+---+----+---+---+---+-----+-------+-------+--------------+---------------+--------------+-------------+-------------+--------------+
|    A Ahmadhel|  3| 3.0|0.0|0.0|0.0|  0.0|2019/20|    BUL|             0|            0.0|           0.0|          0.0|          0.0|           0.0|
|    A Ahmadhel|  1| 1.0|0.0|0.0|0.0|  0.0|   2020|    BUL|             3|            3.0|           0.0|          0.0|          0.0|           0.0|
|    A Ahmadhel|  2| 2.0|0.0|0.0|0.0|  0.0|2020/21|    BUL|             4|            4.0|           0.0|          0.0|          0.0|           0.0|
|    A Ahmadhel|  3| 3.0|0.0|0.0|0.0|  0.0|   2021|    BUL|             6|            6.0|           0.0| 

In [6]:
fielding_data.filter(col("Player") == "Shakib Al Hasan").show()

+---------------+---+----+---+---+---+-----+-------+-------+--------------+---------------+--------------+-------------+-------------+--------------+
|         Player|Mat|Inns|Dis| Ct| St|  D/I| Season|Country|Cumulative Mat|Cumulative Inns|Cumulative Dis|Cumulative Ct|Cumulative St|Cumulative D/I|
+---------------+---+----+---+---+---+-----+-------+-------+--------------+---------------+--------------+-------------+-------------+--------------+
|Shakib Al Hasan|  1| 1.0|0.0|0.0|0.0|  0.0|2006/07|    BAN|             0|            0.0|           0.0|          0.0|          0.0|           0.0|
|Shakib Al Hasan|  6| 6.0|2.0|2.0|0.0|0.333|2007/08|    BAN|             1|            1.0|           0.0|          0.0|          0.0|           0.0|
|Shakib Al Hasan|  1| 1.0|1.0|1.0|0.0|  1.0|2008/09|    BAN|             7|            7.0|           2.0|          2.0|          0.0|          0.29|
|Shakib Al Hasan|  3| 3.0|1.0|1.0|0.0|0.333|   2009|    BAN|             8|            8.0|         

In [7]:
fielding_data = fielding_data.select(['Player', 'Country', 'Season', 'Cumulative Mat', 'Cumulative Inns', 'Cumulative Dis', 'Cumulative Ct', 'Cumulative St', 'Cumulative D/I'])
fielding_data.show(5)

+----------+-------+-------+--------------+---------------+--------------+-------------+-------------+--------------+
|    Player|Country| Season|Cumulative Mat|Cumulative Inns|Cumulative Dis|Cumulative Ct|Cumulative St|Cumulative D/I|
+----------+-------+-------+--------------+---------------+--------------+-------------+-------------+--------------+
|A Ahmadhel|    BUL|2019/20|             0|            0.0|           0.0|          0.0|          0.0|           0.0|
|A Ahmadhel|    BUL|   2020|             3|            3.0|           0.0|          0.0|          0.0|           0.0|
|A Ahmadhel|    BUL|2020/21|             4|            4.0|           0.0|          0.0|          0.0|           0.0|
|A Ahmadhel|    BUL|   2021|             6|            6.0|           0.0|          0.0|          0.0|           0.0|
|A Ahmadhel|    BUL|   2023|             9|            9.0|           0.0|          0.0|          0.0|           0.0|
+----------+-------+-------+--------------+-------------

In [8]:
# calculate null count
from pyspark.sql.functions import isnan, when, count

fielding_data.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in fielding_data.columns]).show()

+------+-------+------+--------------+---------------+--------------+-------------+-------------+--------------+
|Player|Country|Season|Cumulative Mat|Cumulative Inns|Cumulative Dis|Cumulative Ct|Cumulative St|Cumulative D/I|
+------+-------+------+--------------+---------------+--------------+-------------+-------------+--------------+
|     0|      0|     0|             0|              0|             0|            0|            0|             0|
+------+-------+------+--------------+---------------+--------------+-------------+-------------+--------------+



In [9]:
country_codes = {
    'LES': 'Lesotho',
    'BUL': 'Bulgaria',
    'VAN': 'Vanuatu',
    'ROM': 'Romania',
    'Aut': 'Austria',
    'COK': 'Cook Islands',
    'Fran': 'France',
    'SRB': 'Serbia',
    'PAK': 'Pakistan',
    'HUN': 'Hungary',
    'CYP': 'Cyprus',
    'Fiji': 'Fiji',
    'FIN': 'Finland',
    'EST': 'Estonia',
    'CHN': 'China',
    'GRC': 'Greece',
    'CAM': 'Cambodia',
    'GUE': 'Guernsey',
    'SEY': 'Seychelles',
    'JPN': 'Japan',
    'TAN': 'Tanzania',
    'JER': 'Jersey',
    'QAT': 'Qatar',
    'ENG': 'England',
    'UGA': 'Uganda',
    'BER': 'Bermuda',
    'CZK-R': 'Czech Republic',
    'CAY': 'Cayman Islands',
    'IRE': 'Ireland',
    'Mali': 'Mali',
    'BRA': 'Brazil',
    'SUI': 'Switzerland',
    'Peru': 'Peru',
    'Mex': 'Mexico',
    'MOZ': 'Mozambique',
    'Samoa': 'Samoa',
    'HKG': 'Hong Kong',
    'BAN': 'Bangladesh',
    'SL': 'Sri Lanka',
    'PNG': 'Papua New Guinea',
    'ZIM': 'Zimbabwe',
    'GHA': 'Ghana',
    'SWZ': 'Eswatini',  # Swaziland's official name now is Eswatini
    'MYAN': 'Myanmar',
    'IND': 'India',
    'USA': 'United States of America',
    'NEP': 'Nepal',
    'AFG': 'Afghanistan',
    'PAN': 'Panama',
    'NGA': 'Nigeria',
    'SLE': 'Sierra Leone',
    'ESP': 'Spain',
    'Bhm': 'Bahamas',
    'TKY': 'Turkey',
    'MWI': 'Malawi',
    'WI': 'West Indies',
    'IOM': 'Isle of Man',
    'THA': 'Thailand',
    'SWA': 'Eswatini',  # another code for Eswatini
    'SKOR': 'South Korea',
    'GMB': 'Gambia',
    'ISR': 'Israel',
    'KUW': 'Kuwait',
    'Belg': 'Belgium',
    'GER': 'Germany',
    'ITA': 'Italy',
    'CAN': 'Canada',
    'MDV': 'Maldives',
    'Blz': 'Belize',
    'DEN': 'Denmark',
    'INA': 'Indonesia',
    'KENYA': 'Kenya',
    'LUX': 'Luxembourg',
    'STHEL': 'Saint Helena',
    'BHR': 'Bahrain',
    'KSA': 'Saudi Arabia',
    'MLT': 'Malta',
    'Arg': 'Argentina',
    'MNG': 'Mongolia',
    'AUS': 'Australia',
    'GIBR': 'Gibraltar',
    'SGP': 'Singapore',
    'Chile': 'Chile',
    'UAE': 'United Arab Emirates',
    'NZ': 'New Zealand',
    'SCOT': 'Scotland',
    'BHU': 'Bhutan',
    'MAS': 'Malaysia',
    'BOT': 'Botswana',
    'CRC': 'Costa Rica',
    'PHI': 'Philippines',
    'NAM': 'Namibia',
    'RWN': 'Rwanda',
    'OMA': 'Oman',
    'NOR': 'Norway',
    'CRT': 'Croatia',
    'SWE': 'Sweden',
    'Iran': 'Iran',
    'PORT': 'Portugal',
    'NED': 'Netherlands',
    'SA': 'South Africa',
    'SVN': 'Slovenia',
    'GUE': 'Guernsey',
    'MDV': 'Maldives',
    'BHM': 'Bahamas',
    'SWE': 'Sweden',
    'MLT': 'Malta',
    'ITA': 'Italy',
}

# ICC and World teams
icc_world = {
    'ICC/PAK': 'Pakistan',
    'ICC/SL': 'Sri Lanka',
    'ICC/IND': 'India',
    'ICC/NEP': 'Nepal',
    'BAN/ICC': 'Bangladesh',
    'AFG/ICC': 'Afghanistan',
    'SL/World': 'Sri Lanka',
    'SA/World': 'South Africa',
    'AUS/World': 'Australia',
    'BAN/World': 'Bangladesh',
    'WI/World': 'West Indies',
}

# Outlier/Miscellaneous Countries
outlier_countries = {
    '1': 'Miscellaneous Country 1',
    '2': 'Miscellaneous Country 2',
    '3': 'Miscellaneous Country 3',
    'ICC': 'International Cricket Council',
    'World': 'World XI',
}

# Filtered country codes excluding ICC, World teams, and miscellaneous
filtered_countries = {
    code: country
    for code, country in country_codes.items()
    if code not in icc_world and code not in outlier_countries
}
len(country_codes), len(icc_world), len(outlier_countries), len(filtered_countries)

(103, 11, 5, 103)

In [10]:
fielding_data = fielding_data.filter(col('Country').isin(list(filtered_countries.keys())))
fielding_data.show(5)

+----------+-------+-------+--------------+---------------+--------------+-------------+-------------+--------------+
|    Player|Country| Season|Cumulative Mat|Cumulative Inns|Cumulative Dis|Cumulative Ct|Cumulative St|Cumulative D/I|
+----------+-------+-------+--------------+---------------+--------------+-------------+-------------+--------------+
|A Ahmadhel|    BUL|2019/20|             0|            0.0|           0.0|          0.0|          0.0|           0.0|
|A Ahmadhel|    BUL|   2020|             3|            3.0|           0.0|          0.0|          0.0|           0.0|
|A Ahmadhel|    BUL|2020/21|             4|            4.0|           0.0|          0.0|          0.0|           0.0|
|A Ahmadhel|    BUL|   2021|             6|            6.0|           0.0|          0.0|          0.0|           0.0|
|A Ahmadhel|    BUL|   2023|             9|            9.0|           0.0|          0.0|          0.0|           0.0|
+----------+-------+-------+--------------+-------------

In [11]:
fielding_data = fielding_data.replace(filtered_countries,subset=['Country'])
fielding_data.show(5)

+----------+--------+-------+--------------+---------------+--------------+-------------+-------------+--------------+
|    Player| Country| Season|Cumulative Mat|Cumulative Inns|Cumulative Dis|Cumulative Ct|Cumulative St|Cumulative D/I|
+----------+--------+-------+--------------+---------------+--------------+-------------+-------------+--------------+
|A Ahmadhel|Bulgaria|2019/20|             0|            0.0|           0.0|          0.0|          0.0|           0.0|
|A Ahmadhel|Bulgaria|   2020|             3|            3.0|           0.0|          0.0|          0.0|           0.0|
|A Ahmadhel|Bulgaria|2020/21|             4|            4.0|           0.0|          0.0|          0.0|           0.0|
|A Ahmadhel|Bulgaria|   2021|             6|            6.0|           0.0|          0.0|          0.0|           0.0|
|A Ahmadhel|Bulgaria|   2023|             9|            9.0|           0.0|          0.0|          0.0|           0.0|
+----------+--------+-------+--------------+----

In [12]:
fielding_data[['Player','Country']].distinct().count(), fielding_data[['Player']].distinct().count(), fielding_data.distinct().count()

(4156, 4101, 14261)

In [13]:
players_data = utils.load_data(spark,config.PROCESSED_DATA_DIR,'match_players.csv')
players_data = players_data.withColumnRenamed("player", "Player").withColumnRenamed("country", "Country").withColumnRenamed("season", "Season")
print(players_data[['Player',"Country","Season"]].distinct().count())
players_data.show()

[2024-11-24T14:15:09.038+0530] {spark_utils.py:46} INFO - Loading data from match_players.csv.


19449
+---------+---------------+---------+-------+--------+
|  Country|         Player|player_id| Season|match_id|
+---------+---------------+---------+-------+--------+
|Australia|       AJ Finch| b8d490fd|2016/17| 1001349|
|Australia|      M Klinger| b970a03f|2016/17| 1001349|
|Australia|        TM Head| 12b610c2|2016/17| 1001349|
|Australia|   MC Henriques| 32198ae0|2016/17| 1001349|
|Australia|      AJ Turner| ff1e12a0|2016/17| 1001349|
|Australia|    JP Faulkner| 808f425a|2016/17| 1001349|
|Australia|       TD Paine| 5748e866|2016/17| 1001349|
|Australia|     PJ Cummins| ded9240e|2016/17| 1001349|
|Australia|        A Zampa| 14f96089|2016/17| 1001349|
|Australia|     B Stanlake| 6834d1f2|2016/17| 1001349|
|Australia|         AJ Tye| 7c7d63a2|2016/17| 1001349|
|Sri Lanka|    N Dickwella| 45963d9e|2016/17| 1001349|
|Sri Lanka|    WU Tharanga| 7ed9fd56|2016/17| 1001349|
|Sri Lanka| EMDY Munaweera| 5a22d91c|2016/17| 1001349|
|Sri Lanka|  DAS Gunaratne| 770494eb|2016/17| 1001349|
|Sri

In [14]:
fielding_data = fielding_data.join(players_data, ['Player', 'Country', "Season"], 'inner')
fielding_data = fielding_data.select(['player_id', 'Player', 'Country', "Season","Cumulative Mat", "Cumulative Inns", 'Cumulative Dis','Cumulative Ct','Cumulative St','Cumulative D/I'])
print(fielding_data.count())
fielding_data.show(5)

51324


+---------+------------+---------+-------+--------------+---------------+--------------+-------------+-------------+--------------+
|player_id|      Player|  Country| Season|Cumulative Mat|Cumulative Inns|Cumulative Dis|Cumulative Ct|Cumulative St|Cumulative D/I|
+---------+------------+---------+-------+--------------+---------------+--------------+-------------+-------------+--------------+
| b8d490fd|    AJ Finch|Australia|2016/17|            28|           28.0|           7.0|          7.0|          0.0|          0.25|
| b970a03f|   M Klinger|Australia|2016/17|             0|            0.0|           0.0|          0.0|          0.0|           0.0|
| 12b610c2|     TM Head|Australia|2016/17|             4|            4.0|           2.0|          2.0|          0.0|           0.5|
| 32198ae0|MC Henriques|Australia|2016/17|             6|            6.0|           2.0|          2.0|          0.0|          0.33|
| ff1e12a0|   AJ Turner|Australia|2016/17|             0|            0.0|   

In [15]:
fielding_data[['Player','Country']].distinct().count(), fielding_data[['Player']].distinct().count(), fielding_data[['Player','Country',"Season"]].distinct().count()

(3586, 3543, 12096)

In [16]:
utils.spark_save_data(fielding_data, config.PROCESSED_DATA_DIR, 'fielding_data.csv')
spark.stop()

[2024-11-24T14:15:25.208+0530] {spark_utils.py:64} INFO - Successfully wrote data to /usr/ravi/t20/data/2_processedData/fielding_data.csv
